## Deep NLP  1 - BackProp for NNs, DANs, RNNs




### PART A: IMPLEMENTING BACKPROP

In this notebook *backpropagation*  is implemented to train several different neural network architectures. Specifically, the partial derivatives of the provided loss function is computed with respect to each parameter of the network. This is then checked automatically against the output of pytorch's autograd.

**Run the below cell to import pytorch and set up the gradient checking functionality.


In [0]:
import torch
import torch.nn as nn
device = torch.device('cpu')

# checks equality between gradients and those from autograd
# 'params' and 'gradient' are both dictionaries with the same keys, the names of parameters.
# 'params' contains model parameters augmented with pytorch's automatically-computed gradients.  
# 'gradient' contains tensors you calculated yourself.  This function assumes pytorch computed
# them correctly, and checks whether your calculations match.

def gradient_check(params, gradient):
    all_good = True
    for key in params.keys():
        if params[key].grad.size() != gradient[key].size():
            print('GRADIENT ERROR for parameter %s, SIZE ERROR\nyour size: %s\nactual size: %s\n'\
                % (key, gradient[key].size(), 
                   params[key].grad.size()))
            all_good = False
        elif not torch.allclose(params[key].grad, gradient[key], atol=1e-6):
            print('GRADIENT ERROR for parameter %s, VALUE ERROR\nyours: %s\nactual: %s\n'\
                % (key, gradient[key].detach(), 
                   params[key].grad))
            all_good = False
            
    return all_good


### A.1: Single neurons 
The below cell trains a network with just single neurons in each layer on a small dataset of ten examples. The network is defined as:

<center>$\text{h} = \tanh(w_1 * \text{input})$</center>

<center>$\text{pred} = \tanh(w_2 * \text{h})$</center>

Once the partial derivatives $\frac{\partial{L}}{\partial{w_1}}$ and $\frac{\partial{L}}{\partial{w_2}}$ sre implemented correctly, "SUCCESS" message is displayed. 


In [0]:
# initialize model parameters
params = {}
params['w1'] = torch.randn(1, 1, requires_grad=True) # input > hidden with scalar weight w1
params['w2'] = torch.randn(1, 1, requires_grad=True) # hidden > output with scalar weight w2

# set up some training data
inputs = torch.randn(20, 1)
targets = inputs / 2

# training loop
all_good = True
for i in range(len(inputs)):
    
    ## forward prop, then compute loss.
    a = params['w1'] * inputs[i] # intermediate variable, following lecture notes
    hidden = torch.tanh(a)
    b = params['w2'] * hidden
    pred = torch.tanh(b)
    loss = 0.5 * (targets[i] - pred) ** 2 # compute square loss
    loss.backward() # runs autograd
    
    ####################
    # IMPLEMENT BACKPROP: START
    manual_gradient = {}
    manual_gradient['w1'] = torch.zeros(params['w1'].size()) # implement dL/dw1
    manual_gradient['w2'] = torch.zeros(params['w2'].size()) # implement dL/dw2
    
    #Intermediate Computation for w2
    dldo = -1*(targets[i] - pred)
    doda = 1 - pred*pred
    dadw = hidden
    manual_gradient['w2'] = dldo*doda*dadw
    
    #Intermediate Computation for w1
    dadh = params['w2']
    dhdb = (1- hidden*hidden)
    dbdw = inputs[i]
    manual_gradient['w1'] = dldo*doda*dadh*dhdb*dbdw 
   
    # IMPLEMENT BACKPROP: STOP
    ####################

    if not gradient_check(params, manual_gradient):
        all_good = False
        break
    
    # zero gradients after each training example
    params['w1'].grad.zero_()
    params['w2'].grad.zero_() 
    
if all_good:
    print('SUCCESS! you passed the gradient check.')

SUCCESS! you passed the gradient check.


### A.2: Deep averaging network (DAN) for **SENTIMENT ANALYSIS**
DAN is a more complex network featuring multiple matrix/vector operations. Instead of taking single numbers as input, this network takes in *a single sequence of word embeddings* associated with a sentence. Let's call the input $X$; it has dimensionality $N \times D$, where $N$ is the number of words in the sentence and $D$ is the word embedding dimensionality. We'll denote the $i^{th}$ word embedding in the sentence, which corresponds to the $i^{th}$ row of $X$, as $X_i$. The network is trained using softmax / cross entropy loss for **SENTIMENT ANALYSIS**, so each input is associated with a target value $t$ (positive, negative, or neutral). The network is described by the following set of equations:

<center>$\text{ave} = \frac{1}{N} \sum_{i=0}^{N} X_i$</center>

<center>$\text{h} = \text{ReLu}(W_1 \cdot \text{ave})$</center>

<center>$\text{pred} = \text{softmax}(W_2 \cdot h)$</center>

where $\text{ReLu}(x) = max(0, x)$. The defined $\text{softmax}$ derivative is used to compute three partial derivatives: $\frac{\partial{L}}{\partial{W_1}}$, $\frac{\partial{L}}{\partial{W_2}}$, and $\frac{\partial{L}}{\partial{X}}$. 


In [0]:
# Setting HyperParams
N = 4 # all sentences will be of length 4
D = 5 # word embedding dim = 5
M = 3 # hidden dimensionality
labels = {'negative':0, 'neutral':1, 'positive':2}
vocab = {'really':0, 'movie':1, 'was':2, 'good':3, 'not':4, 'okay':5}
num_labels = len(labels)
len_vocab = len(vocab)

# initialize model parameters
params = {}
params['X'] = torch.randn(len_vocab, D, requires_grad=True)
params['W1'] = torch.randn(M, D, requires_grad=True) 
params['W2'] = torch.randn(num_labels, M, requires_grad=True) 

# set up some training data
inputs = [('positive', 'movie was really good'),
          ('neutral', 'really really really okay'),
          ('negative', 'movie was not good')]

# training loop
all_good = True
for i in range(len(inputs)):
    
    # obtain word embeddings for input sentence
    target, sentence = inputs[i]
    target = labels[target]
    input = torch.LongTensor(N)
    for j, w in enumerate(sentence.split()):
        input[j] = vocab[w]
    
    ## forward prop, then compute loss.
    ave = torch.zeros(D) # Compute the word embedding average
    davedx = torch.zeros(len_vocab, D)
    for j in range(N):
        w_idx = input[j]
        ave += params['X'][w_idx]
        davedx[w_idx] = 1.0
    ave = ave / N
    
    # pass it through DAN
    a = torch.mv(params['W1'], ave)
    h = torch.relu(a)
    b = torch.mv(params['W2'], h)
    pred = torch.softmax(b, 0)
    
    loss = -1 * torch.log(pred[target]) # negative log likelihood of target class
    loss.backward() # runs autograd
    
    # Defined derivative for the softmax / CE loss as "dLdb"
    dLdb = pred.clone().detach()
    dLdb[target] -= 1.
    
    #print(ave.size())
    #print(b.size())
    #print(h.size())
    #print(dLdb.size())
    #print(a.size())
    
    ####################
    # IMPLEMENT BACKPROP: START
    your_gradient = {}
    your_gradient['W2'] = torch.zeros(params['W2'].size()) # implement dL/dW2
    your_gradient['W1'] = torch.zeros(params['W1'].size()) # implement dL/dW1
    your_gradient['X'] = torch.zeros(params['X'].size()) # implement dL/dX
    
    your_gradient['W2'] =  torch.ger(dLdb,h)
    
    #Intermediate
    dbdh = params['W2']
    dhda = a > 0
    dadw = ave
  
    prod1 = torch.matmul(dLdb,dbdh)
    prod2 = dhda.float()*prod1
    your_gradient['W1'] = torch.ger(prod2,dadw)
    
    prod3 = torch.mm(dLdb.unsqueeze(0), params["W2"])
    prod4 = prod3 * (a>0).float()
    grad_temp = (1/N) * torch.mm(prod4 ,params['W1'])

    for j in range(N):
      w_idx = input[j]
      your_gradient['X'][w_idx] = your_gradient['X'][w_idx]+grad_temp

    # IMPLEMENT BACKPROP: STOP
    ####################
    
    if not gradient_check(params, your_gradient):
        all_good = False
        break
    
    # zero gradients after each training example
    params['X'].grad.zero_()
    params['W1'].grad.zero_()
    params['W2'].grad.zero_() 
    
if all_good:
    print('SUCCESS! you passed the gradient check.')

SUCCESS! you passed the gradient check.


### A.3: Recurrent neural network (RNN) for **SENTIMENT ANALYSIS**
This implements *backpropagation through time*,  an extension of backpropagation for RNNs, for **SENTIMENT ANALYSIS**. The network architecture is a variant of an RNN with a multiplicative term. The computations in an RNN proceed sequentially, so for inputs where $N=4$, we compute four different hidden states ($h_0, h_1, h_2, h_3$) and feed the final hidden state $h_3$ to the output layer. The network is then defined as:

<center>$h_i= \tanh(W_h \cdot h_{i-1} + W_x \cdot X_i + h_{i-1} * h_{i-2}$)</center>
<center>$\text{pred} = \text{softmax}(W_\text{out} \cdot h_3)$</center>

##### It is helpful to think of an RNN as a feed-forward network by "unrolling" it over the time dimension. 

In [0]:
# initialize model parameters
params = {}
params['X'] = torch.randn(len_vocab, D, requires_grad=True)
params['Wx'] = torch.randn(M, D, requires_grad=True) 
params['Wh'] = torch.randn(M, M, requires_grad=True)
params['Wout'] = torch.randn(num_labels, M, requires_grad=True) 


torch.set_grad_enabled(True)
# training loop
all_good = True
for i in range(len(inputs)):
    
    # obtain word embeddings for input sentence
    target, sentence = inputs[i]
    target = labels[target]
    input = torch.LongTensor(N)
    for j, w in enumerate(sentence.split()):
        input[j] = vocab[w]
    
    ## forward prop, then compute loss.
    hiddens = {} # stores hidden state / intermediate vars at each timestep
    for j in range(N):
        w_idx = input[j]
        hiddens[j] = {}
        
        # no previous hidden state, just project word embedding
        if j == 0:
            hiddens[j]['a'] = torch.mv(params['Wx'], params['X'][w_idx])
            hiddens[j]['a'].retain_grad() #Dont forget to comment
            
        elif j == 1:
            hiddens[j]['a'] = torch.mv(params['Wx'], params['X'][w_idx]) + \
                torch.mv(params['Wh'], hiddens[j-1]['h'])
            hiddens[j]['a'].retain_grad() #Dont forget to comment
            
        else:
            hiddens[j]['a'] = torch.mv(params['Wx'], params['X'][w_idx]) + \
                torch.mv(params['Wh'], hiddens[j-1]['h']) + \
                hiddens[j-1]['h'] * hiddens[j-2]['h']
            hiddens[j]['a'].retain_grad() #Dont forget to comment
            
        hiddens[j]['h'] = torch.tanh(hiddens[j]['a'])
        hiddens[j]['h'].retain_grad()

    
    b = torch.mv(params['Wout'], hiddens[N-1]['h'])
    pred = torch.softmax(b, 0)
    
    loss = -1 * torch.log(pred[target]) # negative log likelihood of target class
    loss.backward() # runs autograd
    
    # Derivative for the softmax / CE loss as "dLdb"
    dLdb = pred.clone().detach()
    dLdb[target] -= 1.
    
    ####################
    # IMPLEMENT BACKPROP: START
    your_gradient = {}
    your_gradient['Wout'] = torch.zeros(params['Wout'].size()) # implement dL/dWout
    your_gradient['X'] = torch.zeros(params['X'].size()) # implement dL/dX
    your_gradient['Wx'] = torch.zeros(params['Wx'].size()) # implement dL/dWx
    your_gradient['Wh'] = torch.zeros(params['Wh'].size()) # implement dL/dWh
    
    #your_gradient['Wout'] 
    #print(dLdb.size())
    #print(hiddens[N-1]['h'].size() )
    #print(params['X'][w_idx].size())
    
    your_gradient['Wout'] = torch.ger(dLdb,hiddens[N-1]['h'])
    dLdhs = torch.zeros((N, M))
    
    for j in range(N - 1, -1, -1):
      if j == N-1:
        dLdhs[j] = torch.mv(params['Wout'].t(), dLdb) 
      #print(dLdhs[j])
      w_idx = input[j]
      dhsdas = (1 - (torch.tanh(hiddens[j]['a'])** 2)) * dLdhs[j]
   
      your_gradient['X'][w_idx] = your_gradient['X'][w_idx] + torch.mv(params['Wx'].t(), dhsdas)
      your_gradient['Wx'] = your_gradient['Wx']+ torch.ger(dhsdas, params['X'][w_idx])
      if j > 0:
        your_gradient['Wh'] = your_gradient['Wh'] + torch.ger(dhsdas, hiddens[j - 1]['h'])
        dLdhs[j - 1] = dLdhs[j - 1] + torch.mv(params['Wh'].t(), dhsdas)
        if j > 1:
          dLdhs[j - 2] = dLdhs[j - 2] + hiddens[j - 1]['h'] * dhsdas
          dLdhs[j - 1] = dLdhs[j - 1] + hiddens[j - 2]['h'] * dhsdas
   
    # IMPLEMENT BACKPROP: STOP
    #############################################
            
    if not gradient_check(params, your_gradient):
        all_good = False
        break
    
    # zero gradients after each training example
    params['X'].grad.zero_()
    params['Wx'].grad.zero_()
    params['Wh'].grad.zero_() 
    params['Wout'].grad.zero_() 

    
if all_good:
    print('SUCCESS! you passed the gradient check.')

SUCCESS! you passed the gradient check.
